<a href="https://colab.research.google.com/github/alvibarra/InroduccionPython/blob/main/etapa_3_de_proyecto_1_opcional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
print('¡Hola! Soy un ChatBot.')
print('Me crearon en 2025 como parte de un ejercicio.')
print('¿Y tu nombre cual es?')
nombre_usuario = str(input())
print('¡Me encanta tu nombre, ' + nombre_usuario + "!")

¡Hola! Soy un ChatBot.
Me crearon en 2025 como parte de un ejercicio.
¿Y tu nombre cual es?
Alvaro I
¡Me encanta tu nombre, Alvaro I!


In [6]:
# instalar la biblioteca gensim en el entorno virtual de Python, porque no esta incluida por default en el entorno de Colab.
# Nota: Al momento de instalar la biblioteca por primera vez surante la sesion se requiere permitir reiniciar la sesión de Colab.

%pip install gensim

In [ ]:
# cargar los componentes (bibliotecas) re, nltk, numpy, y algunos componentes de gensim y sklearn

import re
import nltk
import numpy as np

from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# descargar las bases de datos de nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.corpus import stopwords

# cargar las palabras de español incluidas en la lista de stopwprds

spanish_sw = set(stopwords.words('spanish'))
#print(spanish_sw)

#conectar Google Drive del usuario (de alla vamos a tomar la base de datos de frases célebres)
from google.colab import drive
drive.mount('/content/drive')

#leer el archivo con la base de datos de frases celebres desde Google Drive
path = '/content/drive/MyDrive/Colab Data/'
file = 'frases_celebres_seleccion.txt'

frases = ''
with open(path + file, 'r', errors = 'ignore') as f:
  frases = f.read()
  #print(frases)


In [13]:
 #Uso de la biblioteca nltk para procesar la base de datos de frases célebres
#Para pre-procesar la base de datos de frases célebres vamos a hacer lo siguiente:

#Convertir todo texto a minusculas
#Separar las frases para conformar una lista
#Separar palabras (opcional)
#Solo para fines de referencia vamos a presentar en conteo de frases incluidos, ejemplos de algunos frases, y el número de palabras en la base de datos de frases célebres

# convertir texto plano a lista de frases
onlysent_tokens = nltk.sent_tokenize(frases)
# reemplazar simbolos de nueva linea ocn espacios
onlysent_tokens = [sentence.replace('\n', ' ') for sentence in onlysent_tokens]

# generar una copia de texto con en lowercase
frases_lc = frases.lower()
# convertir texto a lista de palabras
word_tokens = nltk.word_tokenize(frases_lc)

print("\nNúmero de frases célebres en la base de datos:")
print(len(onlysent_tokens))

print("\nEjemplo de frases celebres:")
print(onlysent_tokens[0:3])

print("\nNúmero de palabras en la base de datos de frases célebres:")
print(len(word_tokens))


Número de frases célebres en la base de datos:
862

Ejemplo de frases celebres:
['\ufeffA buen hambre, no hay pan duro.', 'A caballo regalado no le mires el diente.', 'A Dios rogando y con el mazo dando.']

Número de palabras en la base de datos de frases célebres:
10001


In [16]:
# *** preparar datos para su uso en Word2Vec ***
# Preprocesar texto para generar el modelo Word2Vec

# quitar los simbolos de puntuación
# convertir palabras a letras minusculas
# y separar frases en listas de palabras
corpus_sentences = [re.sub(r'[^\w\s]', '', sentence.lower()).split() for sentence in onlysent_tokens]

#print(type(corpus_sentences))
#print(len(corpus_sentences))
#print(corpus_sentences[0:3])

# funcion para eliminar stopwords
def remove_stopwords(sentence_words, sw):
  filtered_words = [word for word in sentence_words if word not in sw]
  return filtered_words

# eliminar stopwords de la base de datos de frases célebres
filtered_sentences = []
for sentence in corpus_sentences:
  filtered_sentence = remove_stopwords(sentence, spanish_sw)
  filtered_sentences.append(filtered_sentence)

#print(type(filtered_sentences))
#print(len(filtered_sentences))
#print(filtered_sentences[0:3])
#*** entrenar el modelo Word2Vec con nuestros datos ***
# Entrenar el modelo Word2Vec
model = Word2Vec(min_count=1,
                     window=6,
                     vector_size=300,
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20,
                     workers=4)

model.build_vocab(filtered_sentences, progress_per=1000)
model.train(filtered_sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)

# Entrenar el modelo Word2Vec (form simplificada)
#model = Word2Vec(sentences=filtered_sentences, vector_size=100, window=5, min_count=1, workers=4)

print("\nCaracteristicas del modelo Word2Vec")
print(model)


Caracteristicas del modelo Word2Vec
Word2Vec<vocab=1942, vector_size=300, alpha=0.03>


In [17]:

#Preguntar al usuario por una frase
user_input = str(input(nombre_usuario + ", introduce su pregunta o algo que te gustaría discutir: "))


# Comparar la frase con la base de datos de frases celebres y buscar la frase celebre con el significado mas cercana a la frase proporcionada por el usuario

# Funcion para construir el vector de una frase
def get_sentence_vector(words, model):
    # determinar el vector de cada palabra en la frase en forma de lista
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    # en caso que vector es vacio generar el vector de ceros y regresarlo
    if not word_vectors:
        return np.zeros(model.vector_size)
    # en caso que vector no es vacio generar un promedio y regresarlo
    return np.mean(word_vectors, axis=0)

# Procesar la pregunta del usuario para construir su vector
user_words = re.sub(r'[^\w\s]', '', user_input.lower()).split()
filtered_user_input = remove_stopwords(user_words, spanish_sw)
user_input_vector = get_sentence_vector(filtered_user_input, model)

print("Pregunta de " + nombre_usuario + ": " + str(user_input))
#print("Pregunta del ususrio filtrada: " + str(filtered_user_input))
#print("Vector del modelo:")
#print(user_input_vector)

# Encontrar la frase mas cercana por el signiicado entre frases célebres
most_similar_sentence = ""
max_similarity = -1

for i, sentence in enumerate(filtered_sentences):
    sentence_vector = get_sentence_vector(sentence, model)
    if np.linalg.norm(user_input_vector) != 0 and np.linalg.norm(sentence_vector) != 0:
        similarity = cosine_similarity([user_input_vector], [sentence_vector])[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_sentence = onlysent_tokens[i]


if not most_similar_sentence:
  print(nombre_usuario + ", lamento, pero no tengo nada que responder en esta ocasión")
else:
  print("La respuesta es: " + most_similar_sentence)


Alvaro I, introduce su pregunta o algo que te gustaría discutir: Es importante ser proactivo
Pregunta de Alvaro I: Es importante ser proactivo
La respuesta es: El ser humano es aquel que no puede ser usado meramente como un medio, sino que debe ser siempre considerado al mismo tiempo como un fin.


In [18]:
#Despedida
print('Hasta la proxima, ' + nombre_usuario + '!')

Hasta la proxima, Alvaro I!
